# Homework 2 : CUAHSI VU : Snow Hydrology

We'll use the setup from the `homework_2_B_setup.ipynb` notebook for this exercise. If you think that you comprised that setup in a previous exercise, just rerun `homework_2_B_setup.ipynb` before continuing.

As before, remember that you want to save your notebooks in a place other than the `examples` directory tree, because files you save there may be deleted and overwritten the next time you start your virtual machine. Save your files somewhere in the `work` directory tree: `work/homework_2` would be a great place for them. So click on "_File --> Save Notebook As..._" and save this notebook as `work/homework_2/homework_2_C_exercise_1.ipynb` before you continue. If things go hopelessly wrong, you can always start over again with the original notebook in your `examples` directory tree.

## C. Homework 2 Exercise 1

This first exercise is mostly just a walk-through that will demonstrate the effects of wind speed on the snow simulations. To do this, we will 
* perform a baseline simulation with the original forcing data, 
* read the original forcing file into an `xarray Dataset`, 
* modify the wind speed for one of the hrus,
* write the new forcings to a new file, and 
* perform a second simulation. 

To make sure that this second simulation uses the new forcing file, we need to modify the SUMMA input file that points to the forcing file to be used (the [forcing file list](https://summa.readthedocs.io/en/latest/input_output/SUMMA_input/#list-of-forcing-files-file)). 

In addition to all this, we will also compare the simulations with observations. We will show an example below. Don't worry, you'll get to do all this yourself in following notebooks.

### Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt # if you want to do some plotting using matplotlib (optional)
from pysumma.Simulation import Simulation # create Simulation objects
from pysumma.hydroshare import hydroshare # create hydroshare objects to upload to HydroShare
from pysumma.LocalParamInfo import LocalParamInfo # modify local parameters.
import numpy as np # provide some mathematical functions
import xarray as xr # read, write, manipulate and analyze NetCDF files
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
resource_id = '05adaf5618ee4dbbbf04195db376cbb4'
case_path = '/home/jovyan/work/{}/{}/data/contents/umpqua'.format(resource_id, resource_id)
filemanager_path = case_path+'/settings/snow_fileManager.txt'
summa_code_path = '/opt/summa'

### Functions

In this section we will pre-define some functions we will use in the rest of the notebook. 

#### Update the forcing file list

The following function will update the forcing file list so that SUMMA will read the right forcing file.

In [ ]:
def update_force_file_list(s, new_force_filename):
    '''Update the forcing file list
       s: SUMMA simulation instance
       new: new forcing filename
       Note this only works if there is a single forcing file'''
    fp = s.setting_path.get_value() + s.forcing_list.get_value()
    with open(fp, 'r') as f:
        lines = f.readlines()
    lines[-1] = "'{}'".format(new_force_filename.split('/')[-1])
    with open(fp, 'w') as f:
        f.writelines(lines)

### Effect of wind speed on snow simulations

We first perform a simulation using the baseline forcing data (this is the same simulation as in `homework_2_B_setup.ipynb`. To do this, we create a SUMMA simulation object and run the model and tage the simulation with the identifier `baseline`.

In [ ]:
S = Simulation(case_data=filemanager_path, summa_code=summa_code_path)
ds_baseline, outfile_baseline = S.execute(run_suffix='baseline', run_option='local')

When we plot the SWE for this simulations, we see that the snow in the clearing and in the forest reach about the same maximum and that there it persists longer in the forest, which is somewhat counter-intuitive:

In [ ]:
ds_baseline.scalarSWE.isel(hru=0).plot(label='Clearing')
ds_baseline.scalarSWE.isel(hru=1).plot(label='Forest')
plt.legend()

Pascal Storck took SWE measurements in the clearing and the forest, which are available to you in `/home/jovyan/work/05adaf5618ee4dbbbf04195db376cbb4/05adaf5618ee4dbbbf04195db376cbb4/data/contents/umpqua/observations/Storck_9697_SWE_obs.nc`. Can you read this file with

In [ ]:
swe_file = '/home/jovyan/work/05adaf5618ee4dbbbf04195db376cbb4/05adaf5618ee4dbbbf04195db376cbb4/data/contents/umpqua/observations/Storck_9697_SWE_obs.nc'
swe_obs = xr.open_dataset(swe_file)

When we plot the observations with the model simulations

In [ ]:
plt.figure(figsize=(10,6))
ds_baseline.scalarSWE.isel(hru=0).plot(label='Clearing -- sim', 
                                       color=cycle[0], linestyle='-')
ds_baseline.scalarSWE.isel(hru=1).plot(label='Forest -- sim',
                                       color=cycle[1], linestyle='-')
swe_obs.SWE.isel(hru=0).plot(label='Clearing -- obs',
                             color=cycle[0], linestyle='--')
swe_obs.SWE.isel(hru=1).plot(label='Forest -- obs',
                             color=cycle[1], linestyle='--')
max_swe = max(ds_baseline.scalarSWE.max(), swe_obs.SWE.max())
plt.ylim(0,max_swe)
plt.legend()

While that does not look too bad for the clearing, the simulations at the forested site don't look right. In the observations we see much less accumulation under the tree canopy, but we do not see this effect in the simulations.

One of the reasons for this is that we are using the same meteorological forcings for both the clearing and forested site. 

Let's take a look at the forcings. To do this, open the forcing file for this simulation and read the file into an `xarray Dataset` object. Note that the name of the forcing file is stored in the forcing list file. The location of that file is

In [ ]:
forcing_file_list = S.file_dir+'/'+S.forcing_list.filename
forcing_file_list

If you look in that file, you will see that the name of the forcing file is `Storck_AboveCanopyPP_9697.nc`. The directory for that file is given by

In [ ]:
S.input_path.filepath

We can use `xarray` to read the file into an `xarray Dataset`.

In [ ]:
force_file_obs  = S.input_path.filepath + '/' + 'Storck_AboveCanopyPP_9697.nc'
force_obs = xr.open_dataset(force_file_obs)

To examine the contents of the file, you can simply type the name of the `Dataset`

In [ ]:
force_obs

We can quickly examine the differences in the forcings between the two hrus by subtracting the forcing data for the two hrus and taking the mean (just showing some xarray features). 

In [ ]:
(force_obs.isel(hru=0)-force_obs.isel(hru=1)).mean()

As you can see, subtracting the two hrus in the `xarra Dataset` will actually subtract all the forcing fields that are in that dataset and the mean difference is 0 for all meteorological variables.

Now keep in mind that SUMMA expects that the forcing variables are specified above the top canopy layer. For most of the forcing variables it may make not much difference whether you take the measurement in a (large) clearing or above the tree canopy (to first order). However, one forcing variable that will definitely be different is the wind speed, which is much lower nearer to the ground in a clearing then above the tree canopy. So using the same wind speed for both simulations is not very realistic. 

Now we can actually estimate the mean difference in the wind speed in the clearing and above the tree canopy by making some assumptions about the wind profile near the surface (see lectures). In this case, it turns out that the wind speed above the forest is about 1.6 times greater than the wind speed in the clearing. So let's adjust the wind speed for hru 1 and rerun the simulations.

To do so, we make a copy of the `force_obs` dataset and will write that to a new input file. Note that in `xarray` you need to use `.copy(deep=True)` when you make the copy. Otherwise the original dataset (`force_obs`) will also be modified when you change your copy (`force_wind`).

In [ ]:
force_wind = force_obs.copy(deep=True)
force_wind['windspd'].loc[{'hru': 1}] *= 1.6

Let's plot the new wind speeds to make sure they now differ

In [ ]:
force_wind['windspd'].isel(hru=0).plot(label='Clearing')
force_wind['windspd'].isel(hru=1).plot(label='Forest')
plt.ylim(0,force_wind['windspd'].max())
plt.legend()

We now want to write this to file and do a new simulation. We first create a new filename for the new file (we replace the '9697' in the original file name by '9697_rescaled_wind')

In [ ]:
force_file_wind = force_file_obs.replace('9697', '9697_rescaled_wind')

We then write the dataset to a NetCDF file (SUMMA expects the file in 'NETCDF3_CLASSIC' format)

In [ ]:
force_wind.to_netcdf(force_file_wind, format='NETCDF3_CLASSIC')

Almost there. The one thing left to do is to update the file with the names of the forcing files, so that SUMMA actually reads the right forcing file. For that we can use the function specified near the start of this notebook (`update_force_file_list()`). The function requires the simulation object and the name of the new forcing file

In [ ]:
update_force_file_list(S, force_file_wind)

OK - now we are all set. Let's do the new simulation with the rescaled wind

In [ ]:
ds_rescaled_wind, out_rescaled_wind = S.execute(run_suffix='rescaled_wind', run_option='local')

And let's replot the simulations and observations

In [ ]:
plt.figure(figsize=(10,6))
ds_rescaled_wind.scalarSWE.isel(hru=0).plot(label='Clearing -- sim', 
                                            color=cycle[0], linestyle='-')
ds_rescaled_wind.scalarSWE.isel(hru=1).plot(label='Forest -- sim',
                                       color=cycle[1], linestyle='-')
swe_obs.SWE.isel(hru=0).plot(label='Clearing -- obs',
                             color=cycle[0], linestyle='--')
swe_obs.SWE.isel(hru=1).plot(label='Forest -- obs',
                             color=cycle[1], linestyle='--')
max_swe = max(ds_rescaled_wind.scalarSWE.max(), swe_obs.SWE.max())
plt.ylim(0,max_swe)
plt.legend()

While that still isn't perfect, these simulations look a lot better. We now get snow interception and increased sublimation from the canopy, which greatly reduces the snow pack below the canopy.

**Keep in mind that the file with the forcing file list now points to the rescaled wind file. If you want to rerun with any other forcing you will need to make sure that the file reported in `forcing_file_list` points to the right input file.**

You can navigate to this file in the file browser to the left.

In addition, the file with the updated forcing will not be overwritten or removed when you rerun the setup in `homework_2_B_setup.ipynb` until you explicitly overwrite it again (for example by rerunning this notebook).

Finally a few hints and things to keep in mind: It is easy to get confused with the model setup and the simulations. If you make changes to the forcings and the simulations do not change, make sure that your simulation object actually reads the right forcing file.

All the forcing files should end up in the directory : `/home/jovyan/work/05adaf5618ee4dbbbf04195db376cbb4/05adaf5618ee4dbbbf04195db376cbb4/
data/contents/umpqua/forcing/`
You can inspect that directory in the file browser to the left (you can also do that in a terminal if you are familiar with unix).

To make sure that the right file is actually being read, you can inspect the `forcing_file_list`, which is at:
`/home/jovyan/work/05adaf5618ee4dbbbf04195db376cbb4/05adaf5618ee4dbbbf04195db376cbb4/
data/contents/umpqua/settings/snow_zForcingFileList.txt`

Keep in mind that commands in a jupyter notebook can be executed in any order, but that that may affect the results. When things don't make sense, re-install the setup for homework 2 (execute the cells in `homework_2_B_setup.ipynb`) and rerun this notebook. To make sure you start afresh, you should also restart the kernel (so there are no old versions in memory). You can do this by going to `Kernel --> Restart Kernel...` and re-executing the notebook cells or selecting `Kernel --> Restart Kernel and Run All Cells`. Normally you would not redo all the model simulations every time, but since the simulations are short, we'll use that method for now.

## On to the next step

That's it for the first part of the second homework as far as the model simulations go. Prof. Lundquist will talk more about the analysis that you need to do on these simulations. You can do the analysis here in this notebook if python is your analysis software of choice or you can upload your output to HydroShare (in NetCDF or csv format) to do additional analysis.

Save this notebook and close the tab. You can also right-click on the file in the left panel if it has a green dot next to it and select "_Shutdown kernel_" from the popup menu to stop the python session that is executing the commands in this notebook. Then open the next notebook: 
`work/homework_2/homework_2_D_exercise_2.ipynb`.